# Novak-Tyson Cell Cycle Model (1993)

_Kevin Siswandi_  
June 2020  

We are going to use the cell cycle model as a test problem for our empirical dynamic modeling method.

In cell cycle model, we have a multi-dimensional system with more than two species. Here, the system can exhibit both **stable oscillations and bistability**. Compare this with the 2-D Yeast Glycolysis model where we have either a stable fixed point or a stable oscillation.

![stable-oscillations](https://www.ncbi.nlm.nih.gov/pmc/articles/instance/1993813/bin/nihms16411f4a.jpg)

[Sible & Tyson (2007) Methods 41:238 - 247](https://pubmed.ncbi.nlm.nih.gov/17189866/). Simulations of the mathematical model of the cell cycle in frog egg extracts describe known and previously unobserved behaviors of the control system: Simulation of the changes in cyclin, preMPF and MPF levels over time in a cycling frog egg extract. The simulations are consistent with a body of experimental data.

<img src="https://www.ncbi.nlm.nih.gov/pmc/articles/instance/1993813/bin/nihms16411f5a.jpg" alt="drawing" width="400"/>

Theoretical predictions from Novak and Tyson: Hysteresis drives mitotic transitions in frog egg extracts. The simulation indicates that the concentration of cyclin required to drive high MPF activity (and thereby enter mitosis) is higher than the concentration required to maintain high MPF activity (and thereby stay in mitosis).

OBSERVE: **The MPF activity jumps down at a much lower level of cyclin than when it jumps up**.

## Basics of the cell cycle

The species involved are:
- Maturation-promoting factor (MPF): a.k.a. mitosis-promoting factor or m-phase promoting factor. This is the activated MPF with one phosphate group removed from cdk1 (relative to pre-MPF).
- pre-MPF. Both activated MPF and pre-MPF consists of cdk-1 and cycB (cyclin).
- cdk1: cyclin-dependent kinase, also called cdc2
- cyclin: is alternately (cyclically) synthesized and degraded over time.
- wee1: opposes MPF activation.

G2 to M phase transition is driven by increase in MPF. One can regulate Cdk/MPF activity via either:
1. synthesis/degradation of cyclin
2. Phosphorylation/dephosphorylation of cdk

**Activation of MPF** by removing an inhibitory phosphate from the cdk subunit. MPF regulation involves positive feedback:
* One of the targets that cdk phosporylates is the protein cdc25, which is a phosphatase. Cdc25 becomes active when it gains a phosphate group.
* One of the targets of Cdc25 is pre-MPF.

Hence mutual activation: MPF activity increases cdc25 activity and vice versa, can lead to bistability.

**Deactivation of MPF** by putting an inhibitory phosphate to the cdk subunit. This occurs through the protein wee1. But MPF inhibits wee1.

![mpf-activation](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fnrc2169/MediaObjects/41568_2007_Article_BFnrc2169_Fig2_HTML.jpg)

[Boutros et al. (2007) Nat Rev Cancer 7, 495-507](https://www.nature.com/articles/nrc2169). a | Cell division cycle 25 (CDC25) phosphatases dephosphorylate and activate cyclin-dependent kinase (CDK)–cyclin complexes, thus allowing catalysis and substrate phosphorylation. WEE1 and MYT1 kinases phosphorylate CDK on tyrosine 15 and threonine 14 of CDK1. Phosphorylation by the CDK-activating kinase (CAK) is required for further activation of CDK–cyclin complexes. For simplicity, an orange P represents T14 and Y15 phosphorylation by WEE1 and MYT1, a blue P represents T161 phosphorylation by CAK. b | Although initial studies suggested a specific role for each CDC25 phosphatase at defined stages of the cell cycle, the current model is that CDC25A, B and C are all involved in phosphorylating CDK–cyclin complexes, such as CDK2–cyclin E at the G1–S transition or CDK1–cyclin B at the entry into mitosis. c | CDC25A, B and C control entry and progression into mitosis. CDC25B is thought to be responsible for the initial activation of CDK1–cyclin B at the centrosome that contributes to microtubule network reorganization and mitotic spindle assembly. Nuclear translocation leads to an auto-amplification process (bold arrows) of CDC25s that then fire the bulk of CDK1–cyclin B complexes and trigger mitosis.

**MPF triggers cyclin degradation**. when MPF activity increases, it activates (through Anaphase Promoting Complex or APC) cyclin protease which degrades cyclin. Thus, MPF positively regulates MPF destruction through APC.

The cell cycle is complex enough; it is almost impossible to make predictions using intuition.

## Structure of the Novak-Tyson model

Main literature: [Novak & Tyson (1993), Journal of Cell Science 106:1153-1168](https://pubmed.ncbi.nlm.nih.gov/8126097/)

The model can be divided into two modules:
1. Cyclin/Cdk dimer regulation
2. Cyclin degradation regulation

Some notable parameters:
* [PPase] represents generic phosphatase activity

In [1]:
def f_novak(x,t,params=None):
    """
    ODEs for Novak-Tyson cell cycle model.
    There are two main classes of equations:
    1. synthesis/degradation of cyclin
    2. phosporylation/dephosporylation
    
    Args:
    x -- array of concentrations for the 7 biochemical species
    t -- time, ignored
    
    Return: a list of derivatives
    """
    if params == None:
        params = {'k1':1, 'k3':0.005,
                  'ka':.02,'Ka':.1,'kb':.1,'Kb':1, 'kc':.13, 'Kc':.01, 'kd':.13, 'Kd':1,
                 'v2_1':, 'v2_2':, 'vwee_1':, 'vwee_2':, 'v25_1':, 'v25_2':,
                 'ke':.02, 'Ke':1, 'kf':, 'Kf':, 'kg':, 'Kg':, 'kh':, 'Kh':,
                 'wee1_total':, 'PPase':, 'CDK_total':, 'cdc25_total':,'IE_total':, 'APC_total':}
    
    cyclin = x[0]
    MPF = x[1]
    preMPF = x[2]
    cdc25P = x[3]
    wee1P = x[4]
    IEP = x[5]
    APC = x[6]
    
    CDK = CDK_total - MPF - preMPF
    
    # cyclin is degraded by APC that is off and on -- overall a weighted combination of the two rates
    k2 = v2_1 * (APC_total - APC) + v2_2 * APC
    
    k25 = v25_1*(cdc25_total - cdc25P) + v25_2*cdc25P
    kwee = vwee_1*wee1P + vwee_2*(wee1_total - wee1P)
    
    # synthesis and degradation of cyclin
    # k1 -- synthesis
    # k2 -- degradation
    # k3 -- dimer formation
    d_cyclin = k1 - k3 * cyclin * CDK - k2 * cyclin
    
    # phosphorylation and dephosphorylation of the CDK subunit via cdc25 (mutual activation) and wee1 (mutual inhibition)
    d_MPF = k3*cyclin*CDK - k2*MPF - kwee*MPF + k25*preMPF
    d_preMPF = -k2*preMPF + kwee*MPF - k25*preMPF
    d_cdc25P = ka*MPF*(cdc25_total - cdc25P)/(Ka + cdc25_total - cdc25P) - kb*PPase*cdc25P/(Kb + cdc25P)
    d_wee1P = ke*MPF*(wee1_total - wee1P)/(Ke + wee1_total - wee1P) - kf*PPase*wee1P/(Kf + wee1P)
    
    d_IEP = kg*MPF*(IE_total - IEP)/(Kg + IE_total - IEP) - kh*PPase*IEP/(Kh + IEP)
    d_APC = kc*IEP*(APC_total - APC)/(Kc + APC_total - APC) - kd*PPase*APC/(Kd + APC)

    return [d_cyclin, d_MPF, d_preMPF, d_cdc25P, d_wee1P, d_IEP, d_APC]

SyntaxError: invalid syntax (<ipython-input-1-d93279037f9b>, line 16)

## Results of the ODE model

1. With the control parameters, the model predicts spontaneuous oscillations of MPF and cyclin. This can be interpreted as rapid cell divisions in newly fertilized oocyte -- consistent with experimental data.
2. Bistability between [Cyclin] total and [MPF] -- using non-degradable cyclin, this is consistent with experimental data.

<img src="image/threshold.png" alt="drawing" width="600"/>

Note that this results use non-degradable cyclin. Otherwise, with normal cyclin, MPF activity goes up, but MPF triggers the degradation of cyclin, so it will go down. The curves show sustained increase of cyclin only because the cyclin is non-degradable.